In [1]:
import sys
sys.path.append('../..')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import numpy as np

In [2]:
import ampligraph
# Benchmark datasets are under ampligraph.datasets module
from ampligraph.datasets import load_fb15k_237
# load fb15k-237 dataset
dataset = load_fb15k_237()

In [3]:
# Import the KGE model
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.datasets import SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY

# Graph loader - loads the data from the file, numpy array, etc and generates batchs for iterating
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                  backend=SQLiteAdapter, # type of backend to use
                                  batch_size=1000,       # batch size to use while iterating over this dataset
                                  dataset_type='train',  # dataset type
                                  use_filter=False,      # Whether to use filter or not
                                  use_indexer=True)      # indicates that the data needs to be mapped to index




In [4]:
# Choose the partitioner - in this case we choose RandomEdges partitioner
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)

# the above code will create a partitioner by passing the graph dataloader object
# the partitioner will partition the data and will internally create multiple graph 
# data loaders for each partition.


_split: memory before: 864.0Bytes, after: 12.825MB, consumed: 12.824MB; exec time: 19.579s


In [5]:
# create and compile a model as usual
partitioned_model = ScoringBasedEmbeddingModel(eta=2, 
                                     k=50, 
                                     scoring_type='DistMult')

partitioned_model.compile(optimizer='adam', loss='multiclass_nll')

partitioned_model.fit(partitioner,            # pass the partitioner object as input to the fit function
                                              # this will generate data for the model during training
                      use_partitioning=True,  # Specify that partitioning needs to be used           
                      epochs=10)              # number of epochs

Epoch 1/10
274/274 [==============================] - 26s 96ms/step - loss: 1095.1127
Epoch 2/10
274/274 [==============================] - 25s 90ms/step - loss: 1090.9618
Epoch 3/10
274/274 [==============================] - 25s 91ms/step - loss: 1080.2867
Epoch 4/10
274/274 [==============================] - 25s 91ms/step - loss: 1052.5120
Epoch 5/10
274/274 [==============================] - 25s 90ms/step - loss: 1004.9535
Epoch 6/10
274/274 [==============================] - 25s 90ms/step - loss: 945.0866
Epoch 7/10
274/274 [==============================] - 25s 90ms/step - loss: 880.6620
Epoch 8/10
274/274 [==============================] - 25s 91ms/step - loss: 817.8426
Epoch 9/10
274/274 [==============================] - 25s 90ms/step - loss: 759.7908
Epoch 10/10
274/274 [==============================] - 25s 90ms/step - loss: 707.4431


In [6]:
dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
            backend=SQLiteAdapter,     # type of backend to use
            batch_size=400,            # batch size to use while iterating over this dataset
            dataset_type='test',       # dataset type
            use_indexer=partitioned_model.data_handler.get_mapper())    # get the mapper from the trained model 
                                                                        # and map the concepts to same indices 
                                                                        # as used during training



28 triples containing invalid keys skipped!


In [7]:
ranks = partitioned_model.evaluate(dataset_loader_test, # pass the dataloader object as input to the 
                                                        # evaluate function. this will generate data
                                                        # for the model during training
                                   batch_size=400)
from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 213s 4s/step


(1171.3521381739897, 0.07391564404652512, 0.0, 0.20427634797925434, 20438)